In [1]:
import wrf_management.run_utilities
import wrf_management.utilities as ut
import importlib
importlib.reload(ut);
import wrf_management.project_global_constants as gc
importlib.reload(gc)
import wrf_management.geogrid as geo
import wrf_management.ungrib as un
importlib.reload(un)
import os
import sqlite3 as sq
import pandas as pd
import wrf_management.base_namelists.base_namelists as bn
importlib.reload(bn);
import f90nml
import subprocess as su

In [2]:
print(gc.RUN_NAME)
job = 'ungrib_press'
file_types=['press']
real = False
hours = 24

LIST_S_LINKS = [
    'ungrib.exe',
    'link_grib.csh',
    'ungrib'
]

LIST_H_LINKS = [
    'Vtable',
    'env_WRFv4.bash'
]

run_2019_02_20


In [3]:
#con = sq.connect(gc.PATH_DB)
gc.PATH_DB

'/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/db_folder/wrf_man.sqlite'

In [4]:
importlib.reload(un)
run_row = wrf_management.run_utilities.get_run_row()
print(run_row)

job_row = wrf_management.run_utilities.get_next_row(job=job)
print(job_row)

wrf_management.run_utilities.update_run_table(val=job_row[job] + 1,
                                              job=job,
                                              date=job_row['date']
                                              )

job_path = wrf_management.run_utilities.getmk_job_path(run_row, job_row, job)
print(job_path)

conf_path = wrf_management.run_utilities.get_conf_path(run_row)
print(conf_path)

type_rows = pd.DataFrame([un.get_type_row(ft, job_row) for ft in file_types])
print(type_rows)

name_list = un.skim_namelist_copy(
    conf_path, job_path, date=job_row.date, prefix=job, hours=hours
)
print(name_list)

index                             1
run_name             run_2019_02_20
geogrid_path                geogrid
config_path          run_2019_02_20
data_path       runs/run_2019_02_20
Name: 0, dtype: object
date              2017-12-02 00:00:00
ungrib_surf                         1
ungrib_press                        0
ungrib_lake                         0
ungrib_avgtsfc                      0
metgrid                             0
real                                0
Name: 0, dtype: object
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2017_12_02/ungrib_press
/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/config_dir/run_2019_02_20
                  date  i  downloaded                      name  untarred  \
0  2017-12-02 00:00:00  1           1  cdas1.20171202.pgrbh.tar         0   

    type  
0  press  
OrderedDict([('share', Namelist([('wrf_core', 'ARW'), ('max_dom', 4), ('start_date', ['2017-12-02_00:00:00', '2017-12-02_00:00:00', '2017-12-02_00:0

In [5]:
if gc.ID=='taito_login':
    wrf_management.run_utilities.copy_hard_links(conf_path, job_path, LIST_H_LINKS)
    wrf_management.run_utilities.copy_soft_links(gc.PATH_WPS, job_path, LIST_S_LINKS)
    importlib.reload(un)
    un.untar_the_files(type_rows, job_path, job_row=job_row)
    
    # in case we need to download the day before
    if job == 'ungrib_press':
        pre_row = wrf_management.run_utilities.get_prev_row(job=job, job_row=job_row)
        trs = pd.DataFrame([un.get_type_row(ft, pre_row) for ft in file_types])
        un.untar_the_files_prev(trs, job_path, job_row=pre_row)

Vtable
env_WRFv4.bash
unilinking
ungrib.exe
unilinking
link_grib.csh
unilinking
ungrib
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/press_tar/cdas1.20171202.pgrbh.tar
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/press_tar/cdas1.20171201.pgrbh.tar
cdas1.t18z.pgrbh01.grib2
cdas1.t18z.pgrbh02.grib2
cdas1.t18z.pgrbh03.grib2
cdas1.t18z.pgrbh04.grib2
cdas1.t18z.pgrbh05.grib2
cdas1.t18z.pgrbh06.grib2
cdas1.t18z.pgrbh07.grib2
cdas1.t18z.pgrbh08.grib2
cdas1.t18z.pgrbh09.grib2
cdas1.t18z.pgrbhanl.grib2


In [ ]:
run_script = \
    """#!/bin/bash
    cd {job_path}
    ./link_grib.csh ./untar/*/*
    source ./env_WRFv4.bash 
    ./ungrib.exe
    exit $?
    """.format(job_path=job_path)
print(run_script)
bs_path = os.path.join(job_path, 'run_me.sh')
bs_file = open(bs_path, 'w')
bs_file.write(run_script)
bs_file.close()

#!/bin/bash
    cd /wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2017_12_02/ungrib_press
    ./link_grib.csh ./untar/*/*
    source ./env_WRFv4.bash 
    ./ungrib.exe
    exit $?
    


In [ ]:
if gc.ID == 'taito_login':
    res = su.run(['/bin/bash', bs_path], stdout=su.PIPE, stderr=su.PIPE)

if gc.ID == 'taito_login' and res.returncode == 0:
    wrf_management.run_utilities.update_run_table(val=100,
                                                  job=job,
                                                  date=job_row['date']
                                                  )